<a href="https://colab.research.google.com/github/patriciacs99/WeaponDetection/blob/main/ObjectDetectionYOLOV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Detección de objetos con YOLOv5 y pytorch

Se van a detectar pistolas

In [5]:
#importar librerías
import os
import shutil
import random
import cv2
import torch
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import glob
import re
from pathlib import Path
import itertools

In [ ]:
!nvidia-smi

Wed Apr  6 08:17:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#conectar con google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#clonar YOLO V5 
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
#cambiar al directorio de YOLO V5
%cd yolov5/
#instalar los requisitos
!pip install -r requirements.txt

In [ ]:
# Descargar modelo con pesos preentrenado
'''
  Modelos a probar:
    - yolov5s.pt
    - yolov5m.pt
    - yolov5l.pt
    - yolov5x.pt
'''
!wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s.pt


In [ ]:
#Rutas
data_yalm = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/PistolsDataset/data.yaml"
pesos_preentrenados = "/content/yolov5/yolov5s.pt"
test_images = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/PistolsDataset/test/images"
pesos_entrenamiento = "/content/yolov5/runs/train/exp/weights/last.pt "
etiquetas_test = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/PistolsDataset/test/labels/*.txt"
etiquetas_detectadas = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/PistolsDataset/Colabs/ResultadosPistolsDataset/Predicciones/detect_results_yolov5s/exp/labels/*.txt"


## Entrenar el modelo

In [ ]:
!python train.py --img 416 --batch 32 --epochs 50 --data data_yalm --weights pesos_preentrenados --nosave --cache

In [ ]:
#Descargar resultados entrenamiento
from google.colab import files

!zip -r train_results_yolov5s.zip /content/yolov5/runs/train/exp
files.download('train_results_yolov5s.zip')

## Analizar los resultados con tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## Inferencia en nuevas imagenes

In [ ]:
!python detect.py --source test_images --weights pesos_entrenamiento --img 416 --save-txt #--save-crop

In [ ]:
#Mostrar las imagenes de test con las detecciones

from IPython.display import Image,display
for imageName in glob.glob("/content/yolov5/runs/detect/exp/*.jpg"):
  display(Image(filename=imageName))
  print("\n")


In [ ]:
#Descargar resultados detecciones
from google.colab import files

!zip -r detect_results_yolov5s.zip /content/yolov5/runs/detect/exp
files.download('detect_results_yolov5s.zip')

# Intersection Over Union


In [ ]:
def intersection_over_union(target_bbox,predicted_bbox):
   
    #formato: x,y,w,h

    #Calcular esquinas bboxes
    box1_x1 = predicted_bbox[0] - predicted_bbox[2]/2
    box1_y1 = predicted_bbox[1] - predicted_bbox[3]/2
    box1_x2 = predicted_bbox[0] + predicted_bbox[2]/2
    box1_y2 = predicted_bbox[1] + predicted_bbox[3]/2
    box2_x1 = target_bbox[0] - target_bbox[2]/2
    box2_y1 = target_bbox[1] - target_bbox[3]/2
    box2_x2 = target_bbox[0] + target_bbox[2]/2
    box2_y2 = target_bbox[1] + target_bbox[3]/2

    #Calcular esquinas interseccion
    x1 = max(box1_x1, box2_x1)
    y1 = max(box1_y1, box2_y1)
    x2 = min(box1_x2, box2_x2)
    y2 = min(box1_y2, box2_y2)

    #area = w * h
    intersection = max(0,(x2 - x1)) * max(0,(y2 - y1))

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-16)


In [ ]:
# Aplicar Intersection Over Union 
detections=[]
detected = sorted(glob.glob(etiquetas_detectadas))
labels = sorted (glob.glob(etiquetas_test))

#Añadir nombre y coordenadas de los objetos de las imagenes de test
for i in range(0,len(labels)):
      file=open(labels[i])
      line = file.readline()
      coord =[]
      detection = ["","",""]
      coord.append(line[1:].split())
      detection[0]="".join(re.findall(r'armas .*_jpg',labels[i]))
      detection[1]=(list(itertools.chain(*coord))) #Aplanar la lista
      detections.append(detection)
      file.close()

#Añadir coordenadas de las detecciones
cont=0
for i in range(0,len(detections)):
    imgName ="".join(re.findall(r'armas .*_jpg',detected[cont]))
    if(imgName == detections[i][0]):
      file=open(detected[cont])
      line = file.readline()
      coord =[]
      coord.append(line[1:].split())
      detections[i][2]= list(itertools.chain(*coord))
      cont = cont+1
      file.close()
    else:
      detections[i][2]=[0]*4

#Convertir valores en floats
for det in detections:
  for i in range(0,4):
     det[1][i] = float(det[1][i])
  for i in range(0,4):
     det[2][i] = float(det[2][i])

#print(*detections,sep="\n")

In [ ]:
#Resultados IoU 
for detection in detections:
  iou = intersection_over_union(detection[1],detection[2])
  print("{}: {:.4f}".format(detection[0],iou))

armas (2101)_jpg: 0.9673
armas (2102)_jpg: 0.6646
armas (2103)_jpg: 0.8511
armas (2104)_jpg: 0.9573
armas (2105)_jpg: 0.9500
armas (2106)_jpg: 0.9659
armas (2107)_jpg: 0.9602
armas (2108)_jpg: 0.9634
armas (2109)_jpg: 0.8799
armas (2110)_jpg: 0.9610
armas (2111)_jpg: 0.9651
armas (2112)_jpg: 0.9523
armas (2113)_jpg: 0.9309
armas (2115)_jpg: 0.9105
armas (2116)_jpg: 0.8181
armas (2117)_jpg: 0.8345
armas (2118)_jpg: 0.5404
armas (2119)_jpg: 0.9527
armas (2120)_jpg: 0.9060
armas (2121)_jpg: 0.5221
armas (2122)_jpg: 0.8902
armas (2123)_jpg: 0.9772
armas (2124)_jpg: 0.9017
armas (2125)_jpg: 0.0000
armas (2126)_jpg: 0.9672
armas (2127)_jpg: 0.0000
armas (2128)_jpg: 0.0000
armas (2129)_jpg: 0.0000
armas (2130)_jpg: 0.8961
armas (2131)_jpg: 0.9037
armas (2132)_jpg: 0.9431
armas (2133)_jpg: 0.8236
armas (2134)_jpg: 0.7082
armas (2135)_jpg: 0.9152
armas (2136)_jpg: 0.5999
armas (2137)_jpg: 0.9615
armas (2138)_jpg: 0.5115
armas (2139)_jpg: 0.9165
armas (2140)_jpg: 0.0000
armas (2141)_jpg: 0.6781
